In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

import utils

In [2]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self, num_layers, num_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(num_neurons, activation='elu', kernel_initializer='he_normal')
                       for _ in range(num_layers)]

    def call(self, inputs, **kwargs):
        a = inputs
        for layer in self.hidden:
            a = layer(a)
        return inputs + a

In [3]:
class ResidualRegressor(keras.models.Model):
    def __init__(self, output_dim, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.hidden_1 = keras.layers.Dense(30, activation='elu', kernel_initializer='he_normal')
        self.block_1 = ResidualBlock(2, 30)
        self.block_2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)

    def call(self, inputs, training=None, mask=None):
        a = self.hidden_1(inputs)
        for _ in range(1 + 3):
            a = self.block_1(a)
        a = self.block_2(a)
        return self.out(a)

In [4]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = utils.load_housing_data()
print(f"x_train.shape = {x_train.shape}, y_train.shape = {y_train.shape}")
print(f"x_val.shape = {x_val.shape}, y_val.shape = {y_val.shape}")
print(f"x_test.shape = {x_test.shape}, y_test.shape = {y_test.shape}")

x_train.shape = (11610, 8), y_train.shape = (11610,)
x_val.shape = (3870, 8), y_val.shape = (3870,)
x_test.shape = (5160, 8), y_test.shape = (5160,)


In [5]:
utils.reset_session()

model = ResidualRegressor(1)
model.compile(loss='mse', optimizer='nadam')
history = model.fit(x_train, y_train, epochs=5)

Epoch 1/5
363/363 [==============================] - 0s 742us/step - loss: 9.1326
Epoch 2/5
363/363 [==============================] - 0s 733us/step - loss: 1.0578
Epoch 3/5
363/363 [==============================] - 0s 789us/step - loss: 0.8870
Epoch 4/5
363/363 [==============================] - 0s 816us/step - loss: 0.5831
Epoch 5/5
363/363 [==============================] - 0s 794us/step - loss: 0.6470


In [6]:
model.save("custom_model.ckpt")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: custom_model.ckpt\assets


In [7]:
utils.reset_session()
model = keras.models.load_model("custom_model.ckpt")
model.evaluate(x_train, y_train)

363/363 [==============================] - 0s 419us/step - loss: 1.0720


1.0719518661499023